In [17]:
# Project 2 of the Udacity Nanodegree - Operationalizing Machine Learning
# The Bank Marketing data will be used - the challenge is to make a determination if the customer will make (or not make) a term deposit
# Gather all imports here
from azureml.core.workspace import Workspace
from azureml.core.compute import ComputeTarget
from azureml.exceptions import ComputeTargetException
from azureml.core.compute.amlcompute import AmlCompute
from azureml.core.experiment import Experiment
from azureml.core.dataset import Dataset
import pandas as pd


In [2]:
# Get the workspace object which will be needed subsequently for most operations
ws = Workspace.from_config()

# Print some basic information from the workdpace as a FYI
print(f'name:{ws.name}, location:{ws.location}\nresource group:{ws.resource_group}, subscription id:{ws.subscription_id}')


name:quick-starts-ws-153563, location:southcentralus
resource group:aml-quickstarts-153563, subscription id:f5091c60-1c3c-430f-8d81-d802f6bf2414


In [4]:
# Next, let's use if it exists, or create if required, a compute cluster to be used by the ML
cc_name = "CPU-CC"  # CPU Compute Cluster

# Access the compute cluster. If it exists, we will have the compute object. If it does not exist, an exception will be thrown upon which the compute cluster can be created
try:
    project_cc = ComputeTarget(workspace=ws, name=cc_name)
    print(f'Compute Cluster target exists and we have a handle to the same')
except ComputeTargetException:
    # Failed to obtain the compute cluster object
    # In all likelihood, a compute cluster of that name has not been created
    # Attempt to create the compute cluster
    # First set up the configuration

    # Specify the configuration of the compute cluster
    cc_cfg = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2', min_nodes=1, max_nodes=6)
    project_cc = ComputeTarget.create(workspace=ws, name=cc_name, provisioning_configuration=cc_cfg)

# At this point - we have access to the compute cluster object. Wait for the compute target to complete provisioing
project_cc.wait_for_completion(show_output='True')



InProgress.....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
# Create an experiment
experiment_name = 'project2-exp'
proj_experiment = Experiment(workspace=ws, name=experiment_name)

In [9]:
proj_experiment

Name,Workspace,Report Page,Docs Page
project2-exp,quick-starts-ws-153563,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
print(f'{proj_experiment}')

Experiment(Name: project2-exp,
Workspace: quick-starts-ws-153563)


In [15]:
# grab the data and create a dataset
# we will use logic similar to the compute target above
# See if the dataset already exists - if so, skip the Dataset creation pieces
# The Bank Marketing data may be found at - https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv
data_uri = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

ds_name = 'Bankdata'
dsets = ws.datasets.keys()

if ds_name in dsets:
    # dataset exists
    proj_ds = dsets[ds_name]
else:
    # Data set not found. Must create it
    proj_ds = Dataset.Tabular.from_delimited_files(data_uri)
    # Register the dataset so tat on repeated runs, teh data does not have to be fethed evey time
    proj_ds.register(workspace=ws, name=ds_name, description='Marketing Bank data')

    

In [18]:
# Take a peek at the data by converting the same to a Pandas dataframe
proj_df = proj_ds.to_pandas_dataframe()

# print the data
proj_df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,...,7,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1,no
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,...,4,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no
